In [1]:
import os, shutil, pathlib
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow import data as tf_data
from tensorflow.keras import layers, models
from tensorflow.keras.utils import image_dataset_from_directory
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
sess = tf.compat.v1.Session()
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Num GPUs Available:  1
Found GPU at: /device:GPU:0


In [13]:
# Load data
new_base_dir = pathlib.Path('/content/drive/My Drive/_images')

# Import training, validation, and test data
train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(480, 480),
    batch_size=8)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(480, 480),
    batch_size=8)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(480, 480),
    batch_size=8)

Found 14211 files belonging to 111 classes.
Found 7050 files belonging to 110 classes.
Found 14081 files belonging to 110 classes.


In [4]:
# Instantiate model
model = Sequential()
model.add(Conv2D(input_shape=(480,480,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

In [5]:
#initialize
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=2, activation="softmax"))

In [6]:
#compile
opt = Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [7]:
# observe model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 480, 480, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 480, 480, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 240, 240, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 240, 240, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 240, 240, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 120, 120, 128)     0         
 g2D)                                                   

In [15]:
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit_generator(steps_per_epoch=100,generator=train_dataset, validation_data=test_dataset, validation_steps=10,epochs=100,callbacks=[checkpoint,early])

Epoch 1/100


<ipython-input-15-e44d8df07d07>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(steps_per_epoch=100,generator=train_dataset, validation_data=test_dataset, validation_steps=10,epochs=100,callbacks=[checkpoint,early])


100/100 [==============================] - ETA: 0s - loss: nan - accuracy: 0.0125

100/100 [==============================] - 156s 1s/step - loss: nan - accuracy: 0.0125 - val_loss: nan - val_accuracy: 0.0125
Epoch 2/100
100/100 [==============================] - ETA: 0s - loss: nan - accuracy: 0.0075

100/100 [==============================] - 102s 1s/step - loss: nan - accuracy: 0.0075 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/100
100/100 [==============================] - ETA: 0s - loss: nan - accuracy: 0.0175

...

100/100 [==============================] - 104s 1s/step - loss: nan - accuracy: 0.0063 - val_loss: nan - val_accuracy: 0.0125
Epoch 5/100
100/100 [==============================] - ETA: 0s - loss: nan - accuracy: 0.0088

100/100 [==============================] - 108s 1s/step - loss: nan - accuracy: 0.0088 - val_loss: nan - val_accuracy: 0.0125
Epoch 6/100
100/100 [==============================] - ETA: 0s - loss: nan - accuracy: 0.0063

100/100 [==============================] - 102s 1s/step - loss: nan - accuracy: 0.0063 - val_loss: nan - val_accuracy: 0.0125
Epoch 7/100
100/100 [==============================] - ETA: 0s - loss: nan - accuracy: 0.0050

100/100 [==============================] - 103s 1s/step - loss: nan - accuracy: 0.0050 - val_loss: nan - val_accuracy: 0.0125
Epoch 8/100
 59/100 [================>.............] - ETA: 41s - loss: nan - accuracy: 0.0042

KeyboardInterrupt: 

In [ ]:
# Fit model to training data
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="A:\Documents\Python Scripts\BirdBot2.0\Training",
        #filepath="/content/drive/My Drive/",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)